<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=200 height=200 align=left class="saturate" >

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Introduction to Machine Learning <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Fall 2022<br>
<font color=3C99D size=5>
    Project <br>
<font color=696880 size=4>
    Project Team 
    
    
____


### Full Name : 
### Student Number : 
___

# Introduction

In this project, we are going to have a brief and elementary hands-on real-world project, predicting breast cancer survival using machine learning models with clinical data and gene expression profiles.

# Data Documentation

For this purpose, we will use "Breast Cancer Gene Expression Profiles (METABRIC)" data. 
The first 31 columns of data contain clinical information including death status.
The next columns of the data contain gene's related information which includes both gene expressions and mutation information. (gene's mutation info columns have been marked with "_mut" at the end of the names of the columns) 
For more information please read the [data documentation](https://www.kaggle.com/datasets/raghadalharbi/breast-cancer-gene-expression-profiles-metabric).

# Data Preparation (15 Points)

In this section you must first split data into three datasets:
<br>
1- clinical dataset
<br>
2- gene expressions dataset
<br>
3- gene mutation dataset. (We will not use this dataset in further steps of the project)

## Data Loading & Splitting

In [1]:
import numpy as np

def suffix_remove(text, suffix='_mut'):
    check_last = suffix + '\n'
    if text[len(text)-len(suffix): len(text)] == suffix or \
        text[len(text)-len(check_last) : len(text)] == check_last:
        return ''
    else:
        return text

file_name = 'METABRIC_RNA_Mutation.csv'

with open (file_name, 'r') as file:
    headers = file.readline()
    headers = np.array([suffix_remove(head) for head in headers.split(',')])
    total_head_num = len(headers)
    headers = headers[np.nonzero(headers)]

dataset = np.loadtxt(file_name, delimiter=',', skiprows=1, dtype='U')

clinical_cols = np.arange(0, 31)
gene_expressions_cols = np.arange(31, len(headers))
gene_mutation_cols = np.arange(len(headers), total_head_num)
clinical_gen_cols = np.arange(0, len(headers))

clinical_dataset = dataset[:, clinical_cols]
gen_expressions_dataset = dataset[:, gene_expressions_cols]
gene_mutation_dataset = dataset[:, gene_mutation_cols]
clinical_gen_dataset = dataset[:, clinical_gen_cols]

clinical_header = headers[0: 31]
gene_expressions_header = headers[31: len(headers)]
clinical_gen_header = headers
categorical_features = ['type_of_breast_surgery', 'cancer_type', 'cancer_type_detailed',
'cellularity', 'pam50_+_claudin-low_subtype', 'er_status_measured_by_ihc', 'er_status', 
'neoplasm_histologic_grade', 'her2_status_measured_by_snp6', 'her2_status', 'tumor_other_histologic_subtype',
'inferred_menopausal_state', 'integrative_cluster', 'primary_tumor_laterality', 'oncotree_code',
'pr_status', '3-gene_classifier_subtype', 'death_from_cancer']



np.savetxt('clinical_dataset.csv', clinical_dataset, delimiter=',', fmt='%s')
np.savetxt('gen_expressions_dataset.csv', gen_expressions_dataset, delimiter=',', fmt='%s')
np.savetxt('gene_mutation_dataset.csv', gene_mutation_dataset, delimiter=',', fmt='%s')
np.savetxt('clinical_gen_dataset.csv', clinical_gen_dataset, delimiter=',', fmt='%s')

## EDA

For each dataset, you must perform a sufficient EDA.

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

df_clinical_gen = pd.read_csv('clinical_gen_dataset.csv', names=clinical_gen_header)


df_clinical = df_clinical_gen.iloc[:, 0:31]
df_gen_expression = df_clinical_gen.iloc[:, 31:]

In [4]:
df_clinical.duplicated().sum()

0

In [5]:
df_gen_expression.duplicated().sum()

0

In [6]:
df_clinical_gen.duplicated().sum()

0

In [7]:
df_clinical.isnull().sum()

patient_id                          0
age_at_diagnosis                    0
type_of_breast_surgery             22
cancer_type                         0
cancer_type_detailed               15
cellularity                        54
chemotherapy                        0
pam50_+_claudin-low_subtype         0
cohort                              0
er_status_measured_by_ihc          30
er_status                           0
neoplasm_histologic_grade          72
her2_status_measured_by_snp6        0
her2_status                         0
tumor_other_histologic_subtype     15
hormone_therapy                     0
inferred_menopausal_state           0
integrative_cluster                 0
primary_tumor_laterality          106
lymph_nodes_examined_positive       0
mutation_count                     45
nottingham_prognostic_index         0
oncotree_code                      15
overall_survival_months             0
overall_survival                    0
pr_status                           0
radio_therap

In [8]:
df_gen_expression.isnull().sum()

brca1      0
brca2      0
palb2      0
pten       0
tp53       0
          ..
tnk2       0
tulp4      0
ugt2b15    0
ugt2b17    0
ugt2b7     0
Length: 489, dtype: int64

In [9]:
df_clinical_gen.isnull().sum()

patient_id                 0
age_at_diagnosis           0
type_of_breast_surgery    22
cancer_type                0
cancer_type_detailed      15
                          ..
tnk2                       0
tulp4                      0
ugt2b15                    0
ugt2b17                    0
ugt2b7                     0
Length: 520, dtype: int64

In [3]:
"""
Handling missing values.
"""

df_clinical_gen.dropna(inplace=True)
# df_clinical_gen.interpolate(method='linear', limit_direction='backward', inplace=True)
df_clinical = df_clinical_gen.iloc[:, 0:31]
df_gen_expression = df_clinical_gen.iloc[:, 31:]
df_clinical.index = [i for i in range(df_clinical.shape[0])]
df_gen_expression.index = [i for i in range(df_clinical.shape[0])]


In [4]:
"""
Handling categorical features in clinical dataset and removing target column and 'death_from_cancer'
from clinical dataframe.
"""
target_name = "overall_survival"
should_remove_name = "death_from_cancer"

target = df_clinical[target_name]

df_clinical.drop(columns=[should_remove_name], axis=1, inplace=True)
df_clinical.drop(columns=[target_name], axis=1, inplace=True)

categorical_features_info = {}
for feature in categorical_features:
    items = []
    for item in df_clinical_gen[feature].unique():
        if pd.notnull(item):
            items.append(item)
    categorical_features_info[feature] = items




encoder = OneHotEncoder(handle_unknown='ignore')
clinical_header_final = [feature for feature in clinical_header if (feature!=target_name and feature!=should_remove_name)]

def apply_onehotencoding(df, categorical_features_info):
    finad_df = pd.DataFrame()
    for feature in clinical_header_final:
        if feature in categorical_features_info.keys():
            encoder_df = pd.DataFrame(encoder.fit_transform(df[[feature]]).toarray())
            cols = [feature+':'+str(category) for category in categorical_features_info[feature]]
            encoder_df.columns = cols
            finad_df[cols] = encoder_df
        else:
            finad_df[feature] = df[[feature]]
    return finad_df
encoded_df_clinical = apply_onehotencoding(df_clinical, categorical_features_info)
encoded_df_clinical.to_csv('encoded_df_clinical.csv', index=False)

## Dimension Reduction (20 + Up to 10 Points Optional)

For each dataset, investigate whether it is needed to use a dimensionality reduction approach or not. If yes, please reduce the dataset's dimension. You can use UMAP for this purpose but any other approach is acceptable. Finding the most important features contains extra points.

In [7]:
import umap
from sklearn.decomposition import PCA
"""
Since most of the features in clinical dataset are categorical (the 18 out of 31) it is
not needed to apply dimensionality reduction on it. But for the gen expression dataset
we deal with many numeric features (489 features); thus a dimensionality reduction process
is required.
"""
n_features_gen = 30
n_features_tot = n_features_gen + 31
pca_reducer = PCA(n_components=n_features_gen)
df_reduced_gen_expression = pca_reducer.fit_transform(df_gen_expression)
df_reduced_gen_expression = pd.DataFrame(df_reduced_gen_expression)

df_reduced_encoded_clinical_gen = pd.concat([encoded_df_clinical, df_reduced_gen_expression], axis=1)


In [8]:
from sklearn.model_selection import train_test_split
def splitter(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train = X_train.to_numpy()
    X_test = X_test.to_numpy()
    y_train = y_train.to_numpy()
    y_test = y_test.to_numpy()
    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = splitter(df_reduced_encoded_clinical_gen, target)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(873, 106)
(219, 106)
(873, 1)
(219, 1)


# Classic Model (25 Points)

In this section, you must implement a classic classification model for clinical, gene expressions, and reduced gene expressions datasets. Using Random Forest is suggested. (minimum acceptable accuracy = 60%)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV


class classical_models():
    def __init__(self, X_train, X_test, y_train, y_test):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.best_params_models_dict = {}

    def hyperparameter_tuning(self, model, random_grid_dic, model_name):
        classifier = RandomizedSearchCV(estimator=model, param_distributions=random_grid_dic, n_iter=1, cv=3, 
        verbose=0, random_state=0, n_jobs=-1)
        classifier.fit(self.X_train, self.y_train.ravel())
        best_params = classifier.best_params_
        self.best_params_models_dict[model_name] = best_params
        return best_params
    
    def random_forest(self):
        random_grid = {'n_estimators': [10*i for i in range(1, 11)],
                    'max_features': ['log2', 'sqrt', None],
                    'max_leaf_nodes': range(5, 10),
                    'max_depth': range(2, 10),
                    'min_samples_split': range(5, 10),
                    'min_samples_leaf': range(0, 4)
                    }
        best_params = self.hyperparameter_tuning(RandomForestClassifier(), random_grid, 'random_forest')
        classifier = RandomForestClassifier(**best_params)
        classifier.fit(self.X_train, self.y_train.ravel())
        y_pred = classifier.predict(self.X_test)
        return y_pred
    
    def extra_tree(self):
        random_grid = {'n_estimators': [10*i for i in range(1, 11)],
            'criterion': ['gini', 'entropy', 'log_loss'],
            'max_features': ['log2', 'sqrt', None],
            'max_leaf_nodes': range(5, 10),
            'max_depth': range(2, 10),
            'min_samples_split': range(5, 10),
            'min_samples_leaf': range(0, 4)
            }
        best_params = self.hyperparameter_tuning(ExtraTreesClassifier(), random_grid, 'extra_tree')
        classifier = ExtraTreesClassifier(**best_params)
        classifier.fit(self.X_train, self.y_train.ravel())
        y_pred = classifier.predict(self.X_test)
        return y_pred

    def decision_tree(self):
        random_grid = {
            'criterion': ['gini', 'entropy', 'log_loss'],
            'max_features': ['log2', 'sqrt', None],
            'max_leaf_nodes': range(5, 10),
            'max_depth': range(2, 10),
            'min_samples_split': range(5, 10),
            'min_samples_leaf': range(0, 4)
            }
        best_params = self.hyperparameter_tuning(DecisionTreeClassifier(), random_grid, 'decision_tree')
        classifier = DecisionTreeClassifier(**best_params)
        classifier.fit(self.X_train, self.y_train.ravel())
        y_pred = classifier.predict(self.X_test)
        return y_pred

    def adaboost(self):
        if self.best_params_models_dict['decision_tree']:
            random_grid = {'n_estimators': [10*i for i in range(1, 11)],
                'learning_rate': [0.001*i for i in range(1, 11)],
                }
            best_params = self.hyperparameter_tuning(AdaBoostClassifier(base_estimator=DecisionTreeClassifier(**self.best_params_models_dict['decision_tree'])),
                                                    random_grid, 'adaboost')
            classifier = AdaBoostClassifier(**best_params)
            classifier.fit(self.X_train, self.y_train.ravel())
            y_pred = classifier.predict(self.X_test)
            return y_pred
        else:
            self.decision_tree()


    def svc(self):
        random_grid = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
            'degree': range(1, 10),
            'gamma': ['scale', 'auto']
            }
        best_params = self.hyperparameter_tuning(SVC(), random_grid, 'svc')
        classifier = SVC(**best_params)
        classifier.fit(self.X_train, self.y_train.ravel())
        y_pred = classifier.predict(self.X_test)
        return y_pred

classical = classical_models(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
y_pred_random_forest = classical.random_forest()
y_pred_extra_tree = classical.extra_tree()
y_pred_decision_tree = classical.decision_tree()
y_pred_adaboost = classical.adaboost()
y_pred_svc = classical.svc()

accuracy_score_dict = {'random_forest': accuracy_score(y_pred_random_forest, y_test),
'extra_tree': accuracy_score(y_pred_extra_tree, y_test),
'decision_tree': accuracy_score(y_pred_decision_tree, y_test),
'adaboost': accuracy_score(y_pred_adaboost, y_test),
'svc': accuracy_score(y_pred_svc, y_test)}

print(accuracy_score_dict)
print(classical.best_params_models_dict)

{'random_forest': 0.7397260273972602, 'extra_tree': 0.776255707762557, 'decision_tree': 0.6118721461187214, 'adaboost': 0.6894977168949772, 'svc': 0.5707762557077626}
{'random_forest': {'n_estimators': 30, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_leaf_nodes': 8, 'max_features': None, 'max_depth': 2}, 'extra_tree': {'n_estimators': 90, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_leaf_nodes': 6, 'max_features': None, 'max_depth': 8, 'criterion': 'log_loss'}, 'decision_tree': {'min_samples_split': 7, 'min_samples_leaf': 2, 'max_leaf_nodes': 6, 'max_features': 'log2', 'max_depth': 3, 'criterion': 'entropy'}, 'adaboost': {'n_estimators': 50, 'learning_rate': 0.005}, 'svc': {'kernel': 'rbf', 'gamma': 'scale', 'degree': 1}}


# Neural Network (30 Points)

In this section, you must implement a neural network model for clinical, gene expressions and reduced gene expressions datasets. Using the MPL models is suggested. (minimum acceptable accuracy = 60%)

In [25]:
from tensorflow import keras
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(alpha=1e-3,
                     hidden_layer_sizes=(20, 15, 10, 5), random_state=1, max_iter=1000)

clf.fit(X_train, y_train.ravel())
y_pred = clf.predict(X_test)
print(accuracy_score(y_pred, y_test))


0.7671232876712328


# Model Comparison (10 Points)

Compare different models and different datasets (clinical, gene expressions, and gene reduced expressions) and try to explain their differences.

#### \# TODO